# Comprehensive Gender Wage Gap Analysis: Balkans Region (2009-2024)

## Focus: North Macedonia & Serbia with Regional Context

This notebook provides comprehensive analysis of gender wage gaps across the Balkan region with special focus on North Macedonia and Serbia.

### Data Sources:
- State Statistical Office of North Macedonia
- Statistical Office of the Republic of Serbia  
- Eurostat
- ILO (International Labour Organization)
- UN Women
- World Bank Gender Data Portal
- Academic research studies

### Analysis Includes:
1. Data validation and cleaning
2. Descriptive statistics and trends
3. Statistical significance testing
4. Regression analysis
5. Forecasting and predictions
6. Comprehensive visualizations

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set visualization styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print("✓ All libraries loaded successfully")
print(f"✓ Pandas version: {pd.__version__}")
print(f"✓ NumPy version: {np.__version__}")

In [ ]:
# Load the expanded dataset
df = pd.read_csv('../data/raw/expanded_balkan_wage_data.csv')

print("="*90)
print("DATASET LOADED")
print("="*90)
print(f"\nTotal records: {len(df)}")
print(f"Columns: {len(df.columns)}")
print(f"\nTime period: {df['year'].min()} - {df['year'].max()}")
print(f"Countries: {df['country'].nunique()}")
print(f"\nCountry breakdown:")
print(df['country'].value_counts().sort_values(ascending=False))

print(f"\nData sources represented:")
print(df['data_source'].value_counts())

print(f"\nFirst few records:")
df.head(10)

## 2. Data Validation and Cleaning

In [ ]:
# Data quality check
print("="*90)
print("DATA QUALITY ASSESSMENT")
print("="*90)

print(f"\n1. MISSING VALUES")
print("-"*90)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Missing'] > 0])
if missing.sum() == 0:
    print("✓ No missing values detected")

print(f"\n2. DATA TYPES")
print("-"*90)
print(df.dtypes)

print(f"\n3. DUPLICATES")
print("-"*90)
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")
if duplicates > 0:
    print("\nRemoving duplicates...")
    df = df.drop_duplicates()
    print(f"✓ Removed {duplicates} duplicates. New size: {len(df)}")

print(f"\n4. VALUE RANGES")
print("-"*90)
print(f"Wage range: {df['avg_monthly_wage'].min():,.0f} - {df['avg_monthly_wage'].max():,.0f}")
print(f"Hours worked range: {df['hours_worked'].min():,.0f} - {df['hours_worked'].max():,.0f}")
print(f"Wage gap range: {df['wage_gap_pct'].min():.2f}% - {df['wage_gap_pct'].max():.2f}%")

print(f"\n5. OUTLIER DETECTION (IQR Method)")
print("-"*90)
Q1 = df['avg_monthly_wage'].quantile(0.25)
Q3 = df['avg_monthly_wage'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['avg_monthly_wage'] < (Q1 - 1.5 * IQR)) | (df['avg_monthly_wage'] > (Q3 + 1.5 * IQR))]
print(f"Outliers detected: {len(outliers)}")
if len(outliers) > 0:
    print("\nOutlier records:")
    print(outliers[['country', 'year', 'gender', 'avg_monthly_wage']])

In [ ]:
# Create clean dataset
df_clean = df.copy()

# Ensure correct data types
df_clean['year'] = df_clean['year'].astype(int)
df_clean['avg_monthly_wage'] = pd.to_numeric(df_clean['avg_monthly_wage'], errors='coerce')
df_clean['hours_worked'] = pd.to_numeric(df_clean['hours_worked'], errors='coerce')
df_clean['wage_gap_pct'] = pd.to_numeric(df_clean['wage_gap_pct'], errors='coerce')

# Sort by country and year
df_clean = df_clean.sort_values(['country', 'year', 'gender'])
df_clean = df_clean.reset_index(drop=True)

print("✓ Data cleaning completed")
print(f"✓ Final dataset size: {len(df_clean)} records")
print(f"✓ Ready for analysis")

## 3. North Macedonia - Comprehensive Analysis

In [ ]:
# Filter North Macedonia data
nm_data = df_clean[df_clean['country'] == 'North Macedonia'].copy()

print("="*100)
print("NORTH MACEDONIA - COMPREHENSIVE GENDER WAGE GAP ANALYSIS")
print("="*100)

print(f"\nDATASET SUMMARY")
print("-"*100)
print(f"Total records: {len(nm_data)}")
print(f"Time span: {nm_data['year'].min()} - {nm_data['year'].max()} ({nm_data['year'].max() - nm_data['year'].min() + 1} years)")
print(f"Unique years: {nm_data['year'].nunique()}")
print(f"Female records: {len(nm_data[nm_data['gender'] == 'Female'])}")
print(f"Male records: {len(nm_data[nm_data['gender'] == 'Male'])}")

# Overall wage statistics
print(f"\n1. OVERALL WAGE STATISTICS BY GENDER")
print("-"*100)
wage_stats = nm_data.groupby('gender')['avg_monthly_wage'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std Dev', 'std'),
    ('Min', 'min'),
    ('Max', 'max'),
    ('25%', lambda x: x.quantile(0.25)),
    ('75%', lambda x: x.quantile(0.75))
]).round(2)
print(wage_stats)

# Calculate overall gender gap
female_mean = nm_data[nm_data['gender'] == 'Female']['avg_monthly_wage'].mean()
male_mean = nm_data[nm_data['gender'] == 'Male']['avg_monthly_wage'].mean()
gap_pct = ((male_mean - female_mean) / male_mean) * 100
gap_abs = male_mean - female_mean

print(f"\n2. OVERALL GENDER WAGE GAP")
print("-"*100)
print(f"Female average wage: {female_mean:,.2f}")
print(f"Male average wage: {male_mean:,.2f}")
print(f"Absolute gap: {gap_abs:,.2f}")
print(f"Percentage gap: {gap_pct:.2f}%")
print(f"Female/Male ratio: {(female_mean/male_mean):.3f}")
print(f"\n→ Women in North Macedonia earn {gap_pct:.2f}% less than men on average")
print(f"→ For every 1.00 unit men earn, women earn {(female_mean/male_mean):.2f} units")

In [ ]:
# Statistical significance testing
print(f"\n3. STATISTICAL SIGNIFICANCE TESTS")
print("-"*100)

female_wages = nm_data[nm_data['gender'] == 'Female']['avg_monthly_wage']
male_wages = nm_data[nm_data['gender'] == 'Male']['avg_monthly_wage']

# T-test
t_stat, p_value = stats.ttest_ind(male_wages, female_wages)
print(f"\nIndependent T-Test:")
print(f"  T-statistic: {t_stat:.4f}")
print(f"  P-value: {p_value:.6f}")
if p_value < 0.001:
    print(f"  Result: HIGHLY SIGNIFICANT (p < 0.001) ***")
elif p_value < 0.01:
    print(f"  Result: VERY SIGNIFICANT (p < 0.01) **")
elif p_value < 0.05:
    print(f"  Result: SIGNIFICANT (p < 0.05) *")
else:
    print(f"  Result: NOT SIGNIFICANT (p >= 0.05)")

# Mann-Whitney U test (non-parametric alternative)
u_stat, u_pvalue = stats.mannwhitneyu(male_wages, female_wages, alternative='two-sided')
print(f"\nMann-Whitney U Test (non-parametric):")
print(f"  U-statistic: {u_stat:.4f}")
print(f"  P-value: {u_pvalue:.6f}")

# Effect size (Cohen's d)
pooled_std = np.sqrt((male_wages.std()**2 + female_wages.std()**2) / 2)
cohens_d = (male_mean - female_mean) / pooled_std
print(f"\nEffect Size (Cohen's d): {cohens_d:.3f}")
if abs(cohens_d) < 0.2:
    effect = "SMALL"
elif abs(cohens_d) < 0.5:
    effect = "MEDIUM"
elif abs(cohens_d) < 0.8:
    effect = "LARGE"
else:
    effect = "VERY LARGE"
print(f"  Interpretation: {effect} effect size")
print(f"\n  → The difference between male and female wages has a {effect.lower()} practical significance")

In [ ]:
# Temporal trend analysis
print(f"\n4. WAGE GAP TREND ANALYSIS OVER TIME")
print("-"*100)

# Calculate yearly gaps
nm_yearly = nm_data.groupby(['year', 'gender'])['avg_monthly_wage'].mean().unstack()
if 'Female' in nm_yearly.columns and 'Male' in nm_yearly.columns:
    nm_yearly['gap_%'] = ((nm_yearly['Male'] - nm_yearly['Female']) / nm_yearly['Male'] * 100).round(2)
    nm_yearly['gap_absolute'] = (nm_yearly['Male'] - nm_yearly['Female']).round(2)
    nm_yearly['female_male_ratio'] = (nm_yearly['Female'] / nm_yearly['Male']).round(3)
    
    print("\nYearly Wage Gap Progression:")
    print(nm_yearly)
    
    # Linear regression for trend
    years = nm_yearly.index.values.reshape(-1, 1)
    gaps = nm_yearly['gap_%'].values
    
    model = LinearRegression()
    model.fit(years, gaps)
    
    trend_slope = model.coef_[0]
    r_squared = r2_score(gaps, model.predict(years))
    
    print(f"\nLinear Regression Results:")
    print(f"  Trend (slope): {trend_slope:.3f} percentage points per year")
    print(f"  R-squared: {r_squared:.3f}")
    print(f"  Intercept: {model.intercept_:.2f}")
    
    if trend_slope < -0.1:
        print(f"\n  ✓ IMPROVING: Wage gap is DECREASING by {abs(trend_slope):.2f}% per year")
    elif trend_slope > 0.1:
        print(f"\n  ✗ WORSENING: Wage gap is INCREASING by {trend_slope:.2f}% per year")
    else:
        print(f"\n  → STABLE: No significant trend (change of {trend_slope:.2f}% per year)")
    
    # Forecast next 3 years
    future_years = np.array([[nm_yearly.index.max() + i] for i in range(1, 4)])
    predictions = model.predict(future_years)
    
    print(f"\n  Forecast for next 3 years:")
    for year, pred in zip(future_years.flatten(), predictions):
        print(f"    {year}: {pred:.2f}%")

In [ ]:
# Sector analysis
print(f"\n5. WAGE GAP BY SECTOR")
print("-"*100)

for sector in nm_data['sector'].unique():
    if sector != 'All':
        sector_data = nm_data[nm_data['sector'] == sector]
        if len(sector_data) > 0:
            female_avg = sector_data[sector_data['gender'] == 'Female']['avg_monthly_wage'].mean()
            male_avg = sector_data[sector_data['gender'] == 'Male']['avg_monthly_wage'].mean()
            gap = ((male_avg - female_avg) / male_avg * 100) if male_avg > 0 else 0
            records = len(sector_data) // 2  # Divided by 2 because we have both genders
            
            print(f"\n{sector} Sector (n={records} observations):")
            print(f"  Female average: {female_avg:,.2f}")
            print(f"  Male average: {male_avg:,.2f}")
            print(f"  Gap: {gap:.2f}%")
            print(f"  Absolute difference: {male_avg - female_avg:,.2f}")

In [ ]:
# Education level analysis
print(f"\n6. WAGE GAP BY EDUCATION LEVEL")
print("-"*100)

for edu in nm_data['education_level'].unique():
    if edu != 'All':
        edu_data = nm_data[nm_data['education_level'] == edu]
        if len(edu_data) > 0:
            female_avg = edu_data[edu_data['gender'] == 'Female']['avg_monthly_wage'].mean()
            male_avg = edu_data[edu_data['gender'] == 'Male']['avg_monthly_wage'].mean()
            gap = ((male_avg - female_avg) / male_avg * 100) if male_avg > 0 else 0
            records = len(edu_data) // 2
            
            print(f"\n{edu} (n={records} observations):")
            print(f"  Female average: {female_avg:,.2f}")
            print(f"  Male average: {male_avg:,.2f}")
            print(f"  Gap: {gap:.2f}%")
            print(f"  Absolute difference: {male_avg - female_avg:,.2f}")

## 4. Serbia - Comprehensive Analysis

In [ ]:
# Filter Serbia data
serbia_data = df_clean[df_clean['country'] == 'Serbia'].copy()

print("="*100)
print("SERBIA - COMPREHENSIVE GENDER WAGE GAP ANALYSIS")
print("="*100)

print(f"\nDATASET SUMMARY")
print("-"*100)
print(f"Total records: {len(serbia_data)}")
print(f"Time span: {serbia_data['year'].min()} - {serbia_data['year'].max()} ({serbia_data['year'].max() - serbia_data['year'].min() + 1} years)")
print(f"Unique years: {serbia_data['year'].nunique()}")
print(f"Female records: {len(serbia_data[serbia_data['gender'] == 'Female'])}")
print(f"Male records: {len(serbia_data[serbia_data['gender'] == 'Male'])}")

# Overall wage statistics
print(f"\n1. OVERALL WAGE STATISTICS BY GENDER")
print("-"*100)
wage_stats_serbia = serbia_data.groupby('gender')['avg_monthly_wage'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std Dev', 'std'),
    ('Min', 'min'),
    ('Max', 'max'),
    ('25%', lambda x: x.quantile(0.25)),
    ('75%', lambda x: x.quantile(0.75))
]).round(2)
print(wage_stats_serbia)

# Calculate overall gender gap
female_mean_serbia = serbia_data[serbia_data['gender'] == 'Female']['avg_monthly_wage'].mean()
male_mean_serbia = serbia_data[serbia_data['gender'] == 'Male']['avg_monthly_wage'].mean()
gap_pct_serbia = ((male_mean_serbia - female_mean_serbia) / male_mean_serbia) * 100
gap_abs_serbia = male_mean_serbia - female_mean_serbia

print(f"\n2. OVERALL GENDER WAGE GAP")
print("-"*100)
print(f"Female average wage: {female_mean_serbia:,.2f}")
print(f"Male average wage: {male_mean_serbia:,.2f}")
print(f"Absolute gap: {gap_abs_serbia:,.2f}")
print(f"Percentage gap: {gap_pct_serbia:.2f}%")
print(f"Female/Male ratio: {(female_mean_serbia/male_mean_serbia):.3f}")
print(f"\n→ Women in Serbia earn {gap_pct_serbia:.2f}% less than men on average")
print(f"→ For every 1.00 unit men earn, women earn {(female_mean_serbia/male_mean_serbia):.2f} units")

In [ ]:
# Statistical significance testing for Serbia
print(f"\n3. STATISTICAL SIGNIFICANCE TESTS")
print("-"*100)

female_wages_serbia = serbia_data[serbia_data['gender'] == 'Female']['avg_monthly_wage']
male_wages_serbia = serbia_data[serbia_data['gender'] == 'Male']['avg_monthly_wage']

if len(female_wages_serbia) > 1 and len(male_wages_serbia) > 1:
    # T-test
    t_stat_serbia, p_value_serbia = stats.ttest_ind(male_wages_serbia, female_wages_serbia)
    print(f"\nIndependent T-Test:")
    print(f"  T-statistic: {t_stat_serbia:.4f}")
    print(f"  P-value: {p_value_serbia:.6f}")
    if p_value_serbia < 0.001:
        print(f"  Result: HIGHLY SIGNIFICANT (p < 0.001) ***")
    elif p_value_serbia < 0.01:
        print(f"  Result: VERY SIGNIFICANT (p < 0.01) **")
    elif p_value_serbia < 0.05:
        print(f"  Result: SIGNIFICANT (p < 0.05) *")
    else:
        print(f"  Result: NOT SIGNIFICANT (p >= 0.05)")
    
    # Effect size
    pooled_std_serbia = np.sqrt((male_wages_serbia.std()**2 + female_wages_serbia.std()**2) / 2)
    cohens_d_serbia = (male_mean_serbia - female_mean_serbia) / pooled_std_serbia
    print(f"\nEffect Size (Cohen's d): {cohens_d_serbia:.3f}")
    if abs(cohens_d_serbia) < 0.2:
        effect_serbia = "SMALL"
    elif abs(cohens_d_serbia) < 0.5:
        effect_serbia = "MEDIUM"
    elif abs(cohens_d_serbia) < 0.8:
        effect_serbia = "LARGE"
    else:
        effect_serbia = "VERY LARGE"
    print(f"  Interpretation: {effect_serbia} effect size")

In [ ]:
# Temporal trend analysis for Serbia
print(f"\n4. WAGE GAP TREND ANALYSIS OVER TIME")
print("-"*100)

serbia_yearly = serbia_data.groupby(['year', 'gender'])['avg_monthly_wage'].mean().unstack()
if 'Female' in serbia_yearly.columns and 'Male' in serbia_yearly.columns:
    serbia_yearly['gap_%'] = ((serbia_yearly['Male'] - serbia_yearly['Female']) / serbia_yearly['Male'] * 100).round(2)
    serbia_yearly['gap_absolute'] = (serbia_yearly['Male'] - serbia_yearly['Female']).round(2)
    serbia_yearly['female_male_ratio'] = (serbia_yearly['Female'] / serbia_yearly['Male']).round(3)
    
    print("\nYearly Wage Gap Progression:")
    print(serbia_yearly)
    
    # Linear regression
    if len(serbia_yearly) > 2:
        years_serbia = serbia_yearly.index.values.reshape(-1, 1)
        gaps_serbia = serbia_yearly['gap_%'].values
        
        model_serbia = LinearRegression()
        model_serbia.fit(years_serbia, gaps_serbia)
        
        trend_slope_serbia = model_serbia.coef_[0]
        r_squared_serbia = r2_score(gaps_serbia, model_serbia.predict(years_serbia))
        
        print(f"\nLinear Regression Results:")
        print(f"  Trend (slope): {trend_slope_serbia:.3f} percentage points per year")
        print(f"  R-squared: {r_squared_serbia:.3f}")
        
        if trend_slope_serbia < -0.1:
            print(f"\n  ✓ IMPROVING: Wage gap is DECREASING by {abs(trend_slope_serbia):.2f}% per year")
        elif trend_slope_serbia > 0.1:
            print(f"\n  ✗ WORSENING: Wage gap is INCREASING by {trend_slope_serbia:.2f}% per year")
        else:
            print(f"\n  → STABLE: No significant trend")

## 5. Comparative Analysis: North Macedonia vs Serbia

In [ ]:
print("="*100)
print("COMPARATIVE ANALYSIS: NORTH MACEDONIA vs SERBIA")
print("="*100)

comparison = pd.DataFrame({
    'Metric': [
        'Data Points',
        'Time Span (years)',
        'Female Avg Wage',
        'Male Avg Wage',
        'Wage Gap %',
        'Absolute Gap',
        'Female/Male Ratio',
        'Std Dev (Combined)'
    ],
    'North Macedonia': [
        len(nm_data),
        nm_data['year'].max() - nm_data['year'].min() + 1,
        female_mean,
        male_mean,
        gap_pct,
        gap_abs,
        female_mean/male_mean,
        nm_data['avg_monthly_wage'].std()
    ],
    'Serbia': [
        len(serbia_data),
        serbia_data['year'].max() - serbia_data['year'].min() + 1,
        female_mean_serbia,
        male_mean_serbia,
        gap_pct_serbia,
        gap_abs_serbia,
        female_mean_serbia/male_mean_serbia,
        serbia_data['avg_monthly_wage'].std()
    ]
})

comparison['Difference (NM - Serbia)'] = comparison['North Macedonia'] - comparison['Serbia']
comparison['% Difference'] = ((comparison['North Macedonia'] - comparison['Serbia']) / comparison['Serbia'] * 100).round(2)

print("\n" + comparison.to_string(index=False))

print("\n" + "="*100)
print("KEY COMPARATIVE INSIGHTS")
print("="*100)

# Gap comparison
if gap_pct > gap_pct_serbia:
    diff = gap_pct - gap_pct_serbia
    print(f"\n1. North Macedonia has a WIDER wage gap ({gap_pct:.2f}%) than Serbia ({gap_pct_serbia:.2f}%)")
    print(f"   Difference: {diff:.2f} percentage points")
else:
    diff = gap_pct_serbia - gap_pct
    print(f"\n1. Serbia has a WIDER wage gap ({gap_pct_serbia:.2f}%) than North Macedonia ({gap_pct:.2f}%)")
    print(f"   Difference: {diff:.2f} percentage points")

# Wage level comparison
print(f"\n2. WAGE LEVELS:")
if female_mean > female_mean_serbia:
    wage_diff = female_mean - female_mean_serbia
    pct_diff = (wage_diff / female_mean_serbia) * 100
    print(f"   • Women in North Macedonia earn {wage_diff:,.2f} ({pct_diff:.1f}%) MORE than women in Serbia")
else:
    wage_diff = female_mean_serbia - female_mean
    pct_diff = (wage_diff / female_mean) * 100
    print(f"   • Women in Serbia earn {wage_diff:,.2f} ({pct_diff:.1f}%) MORE than women in North Macedonia")

if male_mean > male_mean_serbia:
    wage_diff = male_mean - male_mean_serbia
    pct_diff = (wage_diff / male_mean_serbia) * 100
    print(f"   • Men in North Macedonia earn {wage_diff:,.2f} ({pct_diff:.1f}%) MORE than men in Serbia")
else:
    wage_diff = male_mean_serbia - male_mean
    pct_diff = (wage_diff / male_mean) * 100
    print(f"   • Men in Serbia earn {wage_diff:,.2f} ({pct_diff:.1f}%) MORE than men in North Macedonia")

# Data availability
print(f"\n3. DATA AVAILABILITY:")
print(f"   • North Macedonia: {len(nm_data)} data points over {nm_data['year'].nunique()} years")
print(f"   • Serbia: {len(serbia_data)} data points over {serbia_data['year'].nunique()} years")
if len(nm_data) > len(serbia_data):
    print(f"   • North Macedonia has {len(nm_data) - len(serbia_data)} MORE data points ({((len(nm_data)/len(serbia_data)-1)*100):.1f}% more)")
else:
    print(f"   • Serbia has {len(serbia_data) - len(nm_data)} MORE data points ({((len(serbia_data)/len(nm_data)-1)*100):.1f}% more)")